<a href="https://colab.research.google.com/github/vinclaveglia/GraphMBSFinder/blob/master/GraphMBSFinder_19dic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Cloning Repository

In [1]:
! git clone https://github.com/vinclaveglia/GraphMBSFinder.git
!pip install torch-geometric
import sys, os, pathlib
sys.path.insert(0, '/content/GraphMBSFinder')
os.listdir()

Cloning into 'GraphMBSFinder'...
remote: Enumerating objects: 83, done.
remote: Counting objects: 100% (83/83), done.
remote: Compressing objects: 100% (70/70), done.
remote: Total 83 (delta 17), reused 74 (delta 11), pack-reused 0
Receiving objects: 100% (83/83), 616.28 KiB | 7.00 MiB/s, done.
Resolving deltas: 100% (17/17), done.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 6.9 MB/s eta 0:00:00


['.config', 'GraphMBSFinder', 'sample_data']

### Loading libraries and graph neural network

In [2]:
%load_ext autoreload
%autoreload 2

import sys, pathlib
#sys.path.insert(0, '..')
#from base.wp_model import ProteinSegmenter2
#from base.pdb_loader import load_pdb
from base import config
import torch_geometric
import numpy as np
import torch
from base.config import Atom
import pandas as pd
from base.PDB_MBS_preprocessing import get_dataset
from base.input import get_random_train_test_proteins
import warnings
#import biometall
from GCN_and_biometall.gnn_biometall_workflow import *

warnings.filterwarnings('ignore')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

net_CA_path = 'GraphMBSFinder/metadata/model_0.01_50_2_5_0.pth'
net_CB_path = 'GraphMBSFinder/metadata/model_0.01_50_2_5_0.CB.pth_FP0'

net_CA = load_trained_gnn(net_CA_path)
net_CB = load_trained_gnn(net_CB_path)

net_CA.to(device)
net_CB.to(device)

ProteinSegmenter2(
  (conv1): SAGEConv(22, 50, aggr=mean)
  (conv2): SAGEConv(50, 50, aggr=mean)
  (conv3): SAGEConv(50, 50, aggr=mean)
  (conv4): SAGEConv(50, 3, aggr=mean)
)

### Input:

In [3]:
from google.colab import files
uploaded = files.upload()
print(uploaded.keys())
t_pdb=list(uploaded.keys())[0]

Saving 12ca.pdb to 12ca.pdb
dict_keys(['12ca.pdb'])


### Run prediction

In [4]:
path_to_structures = pathlib.Path('.')
path_to_sites = pathlib.Path('.')

db_sites = []

input_graph_CA = get_dataset([t_pdb],
                             atom=Atom.CA,
                             sites=db_sites,
                             path_to_structures=path_to_structures,
                             path_to_sites=path_to_sites)

input_graph_CB = get_dataset([t_pdb],
                             atom=Atom.CB,
                             sites=db_sites,
                             path_to_structures=path_to_structures,
                             path_to_sites=path_to_sites)

input_graph_CA = input_graph_CA[t_pdb]
input_graph_CB = input_graph_CB[t_pdb]

input_graph_CA.to(device)
input_graph_CB.to(device)

pred = gnn_bmtl_predict(t_pdb, input_graph_CA, input_graph_CB, net_CA, net_CB, path_to_structures)

include_and_overlap(pred)

site_lenghts = sorted([int(x) for x in list(pred.keys())], reverse=True)
RES = []
for l in site_lenghts:
    for d in pred[l]:
        if 'included' not in d.keys():
            if 'overlap' not in d.keys():
                print("()",d)
                RES.append(d)
print(RES)

LEN:  255
NON ISOLATI ['HIS_64_A', 'HIS_94_A', 'HIS_96_A', 'GLU_106_A', 'GLU_117_A', 'HIS_119_A', 'ASP_243_A']
UNION ['HIS_64_A', 'HIS_94_A', 'HIS_96_A', 'GLU_106_A', 'GLU_117_A', 'HIS_119_A', 'ASP_243_A', 'HIS_36_A', 'HIS_64_A', 'HIS_94_A', 'HIS_96_A', 'HIS_107_A', 'GLU_117_A', 'HIS_119_A']
- - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - - 
BIOMETALL
() {'residues': ['HIS_94_A', 'HIS_96_A', 'GLU_106_A', 'GLU_117_A', 'HIS_119_A'], 'n_probes': 2, 'center': array([-7.314    , -3.5100002, 15.059    ], dtype=float32)}
[{'residues': ['HIS_94_A', 'HIS_96_A', 'GLU_106_A', 'GLU_117_A', 'HIS_119_A'], 'n_probes': 2, 'center': array([-7.314    , -3.5100002, 15.059    ], dtype=float32)}]


In [ ]:
! pip install nglview


In [9]:
%load_ext autoreload
%autoreload 2

file_path = '4zxn.pdb'
structure = nglview.FileStructure(file_path)
#view = nglview.show_structure_file(file_path)
#view.add_component(structure)
view.add_representation('cartoon', color='grey')
#view.add_representation('line', selection='backbone')

view

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


ValueError: ignored

In [76]:
view.clear_representations()

In [36]:
import nglview as nv

with open("12ca.pdb") as f:
    view = nv.show_file(f, ext="pdb")
    view.clear()
    #view.add_representation('cartoon', color='pink')
    view.add_cartoon(color='light-grey', selection="protein")
    view.remove_cartoon(component=1)
    residue_selection ='200 or 103 or 180'
    view.add_representation('ball+stick', selection=residue_selection, color='green')
    #view.add_representation('stick', selection=residue_selection, color='green')
    coord = [1.0, 2.0, 3.0]
    view.shape.add_sphere(coord, [1, 0, 0], 1.3)
    #view.shape.add_component([{'type': 'sphere', 'position': coord, 'radius': 10.0, 'color': 'blue'}])

    # specify color
    #view.add_cartoon(selection="protein", color='blue')
    # clear representations

    #view.remove_cartoon(component=0)
view

NGLWidget()

In [7]:
import nglview

# Specifica il percorso locale del tuo file PDB
file_path = '12ca.pdb'

# Crea la vista
view = nglview.show_structure_file(file_path)

# Rimuovi tutte le rappresentazioni esistenti
view.clear_representations()

# Aggiungi nuovamente la rappresentazione a cartone con colore verde
view.add_cartoon(color='green')

# Visualizza la vista
view

NGLWidget()

In [12]:
from google.colab import output
output.enable_custom_widget_manager()

In [20]:
import nglview as nv
nv.demo()

NGLWidget()